In [ ]:
import xarray as xr
import xdggs

Download the dataset here: https://zenodo.org/records/10075001

In [ ]:
ds = xr.open_dataset("data/healpix_nolotation.nc")

ds = (
    ds.load()
    .drop_vars(["latitude", "longitude"])
    .stack(cell=["x", "y"], create_index=False)
)

ds.cell_ids.attrs = {
    "grid_name": "healpix",
    "nside": 4096,
    "nest": True,
}

ds

In [ ]:
ds_idx = ds.set_xindex("cell_ids", xdggs.DGGSIndex)

ds_idx

In [ ]:
ds_idx = ds.set_xindex("cell_ids", xdggs.HealpixIndex, nside=4096, nest=True)

ds_idx

In [ ]:
ds_idx.sel(cell_ids=[11320973, 11320975])

In [ ]:
ds_idx.dggs.sel_latlon([48.0, 48.1], -5.0)

In [ ]:
ds2 = ds_idx.dggs.assign_latlon_coords()
ds2

In [ ]:
result = ds_idx.dggs.sel_latlon(ds2.latitude, ds2.longitude)
result

In [ ]:
xr.testing.assert_equal(result.drop_vars(["latitude", "longitude"]), ds)